In [ ]:
import os
import tifftools
import vtk
import numpy
import cv2
import re
import argparse
import pyvista as pv
from os import system, name

path = input("Please enter the path of your tif images >>>> ")
path = path.replace("\\", "/")

dimensions = int(input("Please enter the dimensions of the new tif array (length and width is same number) >>> "))

allImages = input("Do you want to get all images (if yes, type in True otherwise False) >>> ")

finalDirectory = input("Please type the directory of the location you want to save your VTK/I File to >>>> ")
finalDirectory = finalDirectory.replace("\\", "/")
finalName = input("Please type the filename of your VTK/I file >>> ")
extension = input("Please type either vtk or vti (case-sensitive) for your file type >>> ")
finalDirectory = finalDirectory + "/" + finalName

if allImages == "False":
    imageCount = int(input("How many images do you want from the stack? >>> "))

def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(data, key=alphanum_key)

dirlist = sorted_alphanumeric(os.listdir(path))

def PIL2array(img):
    img = cv2.resize(img, (dimensions, dimensions))
    transparency = bool(input("Do you want your image to be transparent? Enter True or False, case sensitive "))
    if transparency == "True":
            # Convert image to image gray
            tmp = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
          
            # Applying thresholding technique
            alpha = cv2.threshold(tmp, 0, 255, cv2.THRESH_BINARY)

            # Using cv2.split() to split channels 
            # of coloured image
            b, g, r = cv2.split(img)

            # Making list of Red, Green, Blue
            # Channels and alpha
            rgba = [b, g, r, alpha]

            # Using cv2.merge() to merge rgba
            # into a coloured/multi-channeled image
            dst = cv2.merge(rgba, 4)
            img = dst
    
    return numpy.array(img, numpy.uint8)

FRAMES = []
FIRST_SIZE = None

if allImages == "True":
    for fn in dirlist:
        img = cv2.imread((os.path.join(path, fn)))
        if FIRST_SIZE is None:
            FIRST_SIZE = img.size
        if img.size == FIRST_SIZE:
            FRAMES.append(PIL2array(img))
            print(fn, ": has been added")
        else:
            print("Discard:", fn, img.size, "<>", FIRST_SIZE)

else:
    imageCount = int(imageCount)
    remove = int(len(dirlist) ) - imageCount
    remove = int(remove)
    boblist = dirlist[ int( (remove/2) )  :  int ( ((remove/2) + imageCount) )]
    for fn in boblist:
        img = cv2.imread((os.path.join(path, fn)))
        if FIRST_SIZE is None:
            FIRST_SIZE = img.size
        if img.size == FIRST_SIZE:
            FRAMES.append(PIL2array(img))
            print(fn, ": has been added")
        else:
              print("Discard:", fn, img.size, "<>", FIRST_SIZE)

print(type(FRAMES))

print()

binary = bool(input("Would you like the image to be binary (1 or 0)? Enter True or False case sensitive"))

if bool == "True":
    FRAMES[FRAMES > 0] = 1


Stack = numpy.dstack(FRAMES)
print(Stack.shape)

data = pv.wrap(Stack)

print(type(data))

j = int(input("Please enter the x and y spacing of the model >>> "))
i = int(input("Please enter the z dimension spacing of the model >>> "))


finDir = finalDirectory
data.spacing = (j, j,i )
print("The wrapping was successful")
dimensions = "Dimensions " + str(j) + "x" + str(j) + "x"+ str(i)
finDir += (dimensions)
finDir += ( "." + extension )
data.save(finDir, binary = True)
print("Your file has been saved to the following location {}".format(finDir))

# If you want to show your data 
# p.show()

In [ ]:
import vtk
import slicer

reader=vtk.vtkXMLImageDataReader()
fileDirec = input("Please enter the directory of your vti file >>> ")
fileDirec = fileDirec.replace("\\", "/")
fileName = input("Please enter the case sensitive filename of your vti file without the extension? >>> ")
fileName += ".vti"
fileDirec = fileDirec + "/" + fileName
print(fileDirec)
reader.SetFileName(fileName)
reader.Update()
imageData = reader.GetOutput()
spacing = imageData.GetSpacing()
origin = imageData.GetOrigin()
imageData.SetOrigin(0,0,0)
imageData.SetSpacing(1,1,1)
volumeNode=slicer.mrmlScene.AddNewNodeByClass("vtkMRMLScalarVolumeNode")
volumeNode.SetAndObserveImageData(imageData)
volumeNode.SetSpacing(spacing)
volumeNode.SetOrigin(origin)
slicer.util.setSliceViewerLayers(volumeNode, fit=True)
print(type(volumeNode))

# Create temporary labelmap
labelVolumeNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLLabelMapVolumeNode")
slicer.vtkSlicerVolumesLogic().CreateLabelVolumeFromVolume(slicer.mrmlScene, labelVolumeNode,volumeNode)

# # Fill temporary labelmap
voxelArray = slicer.util.arrayFromVolume(volumeNode)
labelVoxelArray = slicer.util.arrayFromVolume(labelVolumeNode)
slicer.util.arrayFromVolumeModified(labelVolumeNode)

# # Import labelmap to segmentation
segmentationNode = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLSegmentationNode')
slicer.modules.segmentations.logic().ImportLabelmapToSegmentationNode(labelVolumeNode, segmentationNode)
segmentationNode.CreateClosedSurfaceRepresentation()

volumeNode = segmentationNode
print(type(volumeNode))

direc = input("Please enter the path of where you want to save your OBJ file to >>> ")

slicer.vtkSlicerSegmentationsModuleLogic.ExportSegmentsClosedSurfaceRepresentationToFiles(direc, volumeNode, None, "OBJ" )
# slicer.vtkSlicerSegmentationsModuleLogic.ExportSegmentsClosedSurfaceRepresentationToFiles("stl tests", volumeNode, None, "STL" )
